In [1]:
#importing libraries and packages

import pandas as pd
import numpy as np
import requests #for foursquare API
!pip -q install --upgrade beautifulsoup4 #for web scraping
from bs4 import BeautifulSoup as bs

#Scraping the Wikipedia page for the table containing Postalcode, Borough and Neighborhood information for toronto city
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = bs(website_url, 'lxml')


table = soup.find('table',{'class':'wikitable sortable'})

table_list = table.findAll('td')

#Appending the extrated information into respective lists
boroughs = []
neighborhood = []
postcode = []

for x in range(len(table_list)):
    if x == 0:
        postcode.append(table_list[x].get_text())
       
    elif x%3 == 0:
        postcode.append(table_list[x].get_text())
y=0
for x in range(len(table_list)):
    y += 1
    if x == 1:
        boroughs.append(table_list[x].get_text())
        y=0
    
    if y == 3:
        boroughs.append(table_list[x].get_text())
        y=0

z=0
for x in range(len(table_list)):
    z += 1
    if x == 2:
        neighborhood.append(table_list[x].text[:-1])
        z=0
    
    if z == 3:
        neighborhood.append(table_list[x].text[:-1])
        z=0

#Data cleaning #1: Removing rows that have boroughs with 'Not assigned'
for s in range(len(boroughs)-1,-1,-1):
    if boroughs[s] == 'Not assigned':
        del boroughs[s]
        del postcode[s]
        del neighborhood[s]
        
#Data cleaning #2: Giving neighborhood cells with'Not assigned' value the same value as its respective Borough      
for s in range(len(neighborhood)):
    if neighborhood[s] == 'Not assigned':
        neighborhood[s] = boroughs[s]
        
#Entering all information into a dataframe
column_names = ['Postalcode', 'Borough', 'Neighborhood'] 

df_neighborhoods = pd.DataFrame(columns=column_names)
df_neighborhoods['Postalcode']=postcode
df_neighborhoods['Borough']=boroughs
df_neighborhoods['Neighborhood']=neighborhood

#Grouping rows that have the same Postalcode
df_neighborhood_grouped=df_neighborhoods.groupby(['Postalcode','Borough'],as_index=False).agg(lambda x: ", ".join(x))

#Reading geospatial data for toronto and merging it with previous dataframe
toronto_data = pd.read_csv('https://cocl.us/Geospatial_data')

df_neighborhood_grouped['Latitude'] = toronto_data['Latitude']
df_neighborhood_grouped['Longitude'] = toronto_data['Longitude']
df_neighborhood_grouped


,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  52.92 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  34.69 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  36.65 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  47.70 MB/s
Libraries imported.


In [3]:
#getting geographical coordinates of toronto area

address = 'Toronto, CAN'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6607002, -79.3850889.


In [4]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_neighborhood_grouped['Latitude'], df_neighborhood_grouped['Longitude'], df_neighborhood_grouped['Borough'], df_neighborhood_grouped['Neighborhood']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [5]:
#We are going to create a k-means cluster map that clusters all nearby boroughs together and indicate the neighborhoods in those clusters

#create a dataframe with only latitude and longitude values for cluster analysis
toronto_grouped_cluster = df_neighborhood_grouped[['Latitude', 'Longitude']]

#set number of cluster and initiate cluster analysis
kclusters = 5
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 2, 0, 2], dtype=int32)

In [6]:

# add clustering labels
df_neighborhood_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

df_neighborhood_grouped.tail()

,Cluster Labels,Postalcode,Borough,Neighborhood,Latitude,Longitude
98,1,M9N,York,Weston,43.706876,-79.518188
99,1,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,1,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,1,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
102,1,M9W,Etobicoke,Northwest,43.706748,-79.594054


In [7]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, neigh, cluster in zip(df_neighborhood_grouped['Latitude'], df_neighborhood_grouped['Longitude'], df_neighborhood_grouped['Borough'], df_neighborhood_grouped['Neighborhood'], df_neighborhood_grouped['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster) + ' | '+' Borough - '+ str(poi) + ' | ' + ' Neighborhood(s) - ' + str(neigh), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [8]:
CLIENT_ID = 'FDXBNFE5MFOKUZYSQRGPZEMKVFMYVWV2CFMYFI4DHXGEBRMO' # your Foursquare ID
CLIENT_SECRET = 'ECL1USAPB3XU33EGL23N15RUJ1DU14WDXIIUHZJ431FDZFUV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FDXBNFE5MFOKUZYSQRGPZEMKVFMYVWV2CFMYFI4DHXGEBRMO
CLIENT_SECRET:ECL1USAPB3XU33EGL23N15RUJ1DU14WDXIIUHZJ431FDZFUV


In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postalcode', 
                  'Neighborhood(s) Latitude', 
                  'Neighborhood(s) Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    #Retain information about type of restaurant only
    nearby_venues=nearby_venues[nearby_venues['Venue Category'].str.contains('Restaurant')]
    nearby_venues=nearby_venues.reset_index(drop=True)
    
    return(nearby_venues)

In [10]:
#calling to Foursquare API
LIMIT=100
toronto_venues = getNearbyVenues(names=df_neighborhood_grouped['Postalcode'], #as neighborhoods have numerous values, we will use Postalcode as our key to generate results since it is a singular unique value
                                   latitudes=df_neighborhood_grouped['Latitude'],
                                   longitudes=df_neighborhood_grouped['Longitude']
                                  )



M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X
M2H
M2J
M2K
M2L
M2M
M2N
M2P
M2R
M3A
M3B
M3C
M3H
M3J
M3K
M3L
M3M
M3N
M4A
M4B
M4C
M4E
M4G
M4H
M4J
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5M
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6A
M6B
M6C
M6E
M6G
M6H
M6J
M6K
M6L
M6M
M6N
M6P
M6R
M6S
M7A
M7R
M7Y
M8V
M8W
M8X
M8Y
M8Z
M9A
M9B
M9C
M9L
M9M
M9N
M9P
M9R
M9V
M9W


In [31]:
print(toronto_venues.shape)
toronto_venues


(518, 7)


,Postalcode,Neighborhood(s) Latitude,Neighborhood(s) Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1E,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
2,M1G,43.770992,-79.216917,Korean Grill House,43.770812,-79.214502,Korean Restaurant
3,M1H,43.773136,-79.239476,Federick Restaurant,43.774697,-79.241142,Hakka Restaurant
4,M1H,43.773136,-79.239476,Drupati's Roti & Doubles,43.775222,-79.241678,Caribbean Restaurant
5,M1H,43.773136,-79.239476,Thai One On,43.774468,-79.241268,Thai Restaurant
6,M1L,43.711112,-79.284577,Dairy Queen,43.710378,-79.290701,Fast Food Restaurant
7,M1M,43.716316,-79.239476,Vincent's Spot,43.717002,-79.242353,American Restaurant
8,M1P,43.757410,-79.273304,Kairali,43.754768,-79.277199,Indian Restaurant
9,M1P,43.757410,-79.273304,Kim Kim restaurant,43.753833,-79.276611,Chinese Restaurant


In [13]:
toronto_venues.groupby('Postalcode').count()

,Neighborhood(s) Latitude,Neighborhood(s) Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postalcode,,,,,,
M1B,1,1,1,1,1,1
M1E,1,1,1,1,1,1
M1G,1,1,1,1,1,1
M1H,3,3,3,3,3,3
M1L,1,1,1,1,1,1
M1M,1,1,1,1,1,1
M1P,5,5,5,5,5,5
M1R,2,2,2,2,2,2
M1S,1,1,1,1,1,1


In [14]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 51 uniques categories.


In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")


# add neighborhood column back to dataframe
toronto_onehot['Postalcode'] = toronto_venues['Postalcode'] 



# move Postalcode column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]



toronto_onehot.head(10)

,Postalcode,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Dim Sum Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hakka Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,New American Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Southern / Soul Food Restaurant,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,M1B,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M1E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M1G,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M1H,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M1H,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,M1H,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
6,M1L,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,M1M,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,M1P,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,M1P,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
toronto_onehot.shape

(518, 52)

In [19]:
toronto_grouped = toronto_onehot.groupby('Postalcode').mean().reset_index() #Create frequency
toronto_grouped

,Postalcode,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Dim Sum Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hakka Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,New American Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Southern / Soul Food Restaurant,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,M1B,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
1,M1E,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
2,M1G,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,1.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
3,M1H,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.333333,0.000000,0.000000,0.000000
4,M1L,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
5,M1M,0.000000,1.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
6,M1P,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,0.000000,0.000000,0.000000,0.00,0.000000,0.200000,0.00,0.000000,0.000000,0.000000,

In [20]:
toronto_grouped.shape

(69, 52)

In [22]:
num_top_venues = 5

#Making the top 5 most common restaurant cuisines
for hood in toronto_grouped['Postalcode']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Postalcode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                   venue  freq
0   Fast Food Restaurant   1.0
1      Afghan Restaurant   0.0
2  Indonesian Restaurant   0.0
3    Japanese Restaurant   0.0
4      Jewish Restaurant   0.0


----M1E----
                     venue  freq
0       Mexican Restaurant   1.0
1        Afghan Restaurant   0.0
2  New American Restaurant   0.0
3      Japanese Restaurant   0.0
4        Jewish Restaurant   0.0


----M1G----
                     venue  freq
0        Korean Restaurant   1.0
1        Afghan Restaurant   0.0
2  New American Restaurant   0.0
3      Japanese Restaurant   0.0
4        Jewish Restaurant   0.0


----M1H----
                     venue  freq
0         Hakka Restaurant  0.33
1          Thai Restaurant  0.33
2     Caribbean Restaurant  0.33
3        Afghan Restaurant  0.00
4  New American Restaurant  0.00


----M1L----
                   venue  freq
0   Fast Food Restaurant   1.0
1      Afghan Restaurant   0.0
2  Indonesian Restaurant   0.0
3    Japanese Restaurant   0

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postalcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Restaurant'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Restaurant'.format(ind+1))

# create a new dataframe
neighborhoods_restaurants_sorted = pd.DataFrame(columns=columns)
neighborhoods_restaurants_sorted['Postalcode'] = toronto_grouped['Postalcode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_restaurants_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_restaurants_sorted

,Postalcode,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant
0,M1B,Fast Food Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant,Eastern European Restaurant
1,M1E,Mexican Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
2,M1G,Korean Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
3,M1H,Thai Restaurant,Caribbean Restaurant,Hakka Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant
4,M1L,Fast Food Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant,Eastern European Restaurant
5,M1M,American Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
6,M1P,Indian Restaurant,Latin American Restaurant,Chinese Restaurant,Vietnamese Restaurant,Tapas Restaurant,Cuban Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
7,M1R,Middle Eastern Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
8,M1S,Chinese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
9,M1T,Thai Restaurant,Italian Restaurant,Fast Food Restaurant,Chinese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant


In [27]:
toronto_merged = df_neighborhood_grouped

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_restaurants_sorted.set_index('Postalcode'), on='Postalcode')

toronto_merged 

,Cluster Labels,Postalcode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,Fast Food Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant,Eastern European Restaurant
1,0,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Mexican Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
3,0,M1G,Scarborough,Woburn,43.770992,-79.216917,Korean Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
4,0,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Thai Restaurant,Caribbean Restaurant,Hakka Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant
5,0,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Fast Food Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant,Eastern European Restaurant
8,0,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,American Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
9,2,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0] #data in cluster 0

,Cluster Labels,Postalcode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,Fast Food Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant,Eastern European Restaurant
1,0,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Mexican Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
3,0,M1G,Scarborough,Woburn,43.770992,-79.216917,Korean Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
4,0,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Thai Restaurant,Caribbean Restaurant,Hakka Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant
5,0,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,American Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
10,0,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,Indian Restaurant,Latin American Restaurant,Chinese Restaurant,Vietnamese Restaurant,Tapas Restaurant,Cuban Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
11,0,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849,Middle Eastern Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant


In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1] #data in cluster 1

,Cluster Labels,Postalcode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant
31,1,M3L,North York,Downsview West,43.739015,-79.506944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,1,M3M,North York,Downsview Central,43.728496,-79.495697,Korean Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
33,1,M3N,North York,Downsview Northwest,43.761631,-79.520999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,1,M6L,North York,"Downsview, North Park, Upwood Park",43.713756,-79.490074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,1,M6M,York,"Del Ray, Keelesdale, Mount Dennis, Silverthorn",43.691116,-79.476013,Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
81,1,M6N,York,"The Junction North, Runnymede",43.673185,-79.487262,Caribbean Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
84,1,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,Italian Restaurant,Sushi Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Falafel Restaurant,Restaurant,Latin American Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Filipino Restaurant
86,1,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819,American Restaurant,Mediterranean Restaurant,Middle Eastern Restaurant,Vietnamese Restaurant,Doner Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
88,1,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,American Restaurant,Fast Food Restaurant,Seafood Restaurant,Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant
89,1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2] #data in cluster 2

,Cluster Labels,Postalcode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant
7,2,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Fast Food Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant,Eastern European Restaurant
9,2,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,2,M3A,North York,Parkwoods,43.753259,-79.329656,Fast Food Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant,Eastern European Restaurant
26,2,M3B,North York,Don Mills North,43.745906,-79.352188,Japanese Restaurant,Caribbean Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
27,2,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923,Asian Restaurant,Italian Restaurant,Japanese Restaurant,Fast Food Restaurant,Restaurant,Chinese Restaurant,Dim Sum Restaurant,Vietnamese Restaurant,Dumpling Restaurant,German Restaurant
34,2,M4A,North York,Victoria Village,43.725882,-79.315572,Portuguese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
35,2,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,Fast Food Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant,Eastern European Restaurant
36,2,M4C,East York,Woodbine Heights,43.695344,-79.318389,Asian Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
37,2,M4E,East Toronto,The Beaches,43.676357,-79.293031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,2,M4G,East York,Leaside,43.709060,-79.363452,Sushi Restaurant,Restaurant,Mexican Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant


In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3] #data in cluster 3

,Cluster Labels,Postalcode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant
47,3,M4S,Central Toronto,Davisville,43.704324,-79.388790,Sushi Restaurant,Italian Restaurant,Indian Restaurant,Greek Restaurant,Thai Restaurant,Seafood Restaurant,Restaurant,Japanese Restaurant,Dim Sum Restaurant,Filipino Restaurant
48,3,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,3,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,Vietnamese Restaurant,American Restaurant,Sushi Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
50,3,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,3,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,Restaurant,Italian Restaurant,Chinese Restaurant,Caribbean Restaurant,Japanese Restaurant,Indian Restaurant,Taiwanese Restaurant,Thai Restaurant,American Restaurant,Filipino Restaurant
52,3,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,Japanese Restaurant,Sushi Restaurant,Restaurant,Fast Food Restaurant,Mediterranean Restaurant,Vietnamese Restaurant,Mexican Restaurant,American Restaurant,Caribbean Restaurant,Chinese Restaurant
53,3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,Restaurant,Mexican Restaurant,French Restaurant,Italian Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,German Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
54,3,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,Middle Eastern Restaurant,Thai Restaurant,Italian Restaurant,Japanese Restaurant,Restaurant,Ramen Restaurant,Caribbean Restaurant,Ethiopian Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant
55,3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,Restaurant,Italian Restaurant,American Restaurant,Japanese Restaurant,Seafood Restaurant,Indian Restaurant,New American Restaurant,Latin American Restaurant,Fast Food Restaurant,Middle Eastern Restaurant
56,3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,Seafood Restaurant,Restaurant,Comfort Food Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Greek Restaurant,Thai Restaurant,Belgian Restaurant,Brazilian Restaurant


In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4] #data in cluster 4

,Cluster Labels,Postalcode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant
17,4,M2H,North York,Hillcrest Village,43.803762,-79.363452,Fast Food Restaurant,Mediterranean Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant
18,4,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Fast Food Restaurant,Asian Restaurant,American Restaurant,Japanese Restaurant,Restaurant,Vietnamese Restaurant,Doner Restaurant,German Restaurant,French Restaurant,Filipino Restaurant
19,4,M2K,North York,Bayview Village,43.786947,-79.385975,Japanese Restaurant,Chinese Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
20,4,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,4,M2N,North York,Willowdale South,43.770120,-79.408493,Restaurant,Ramen Restaurant,Vietnamese Restaurant,Fast Food Restaurant,Indonesian Restaurant,Sushi Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Cuban Restaurant,Filipino Restaurant
23,4,M2P,North York,York Mills West,43.752758,-79.400049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,4,M2R,North York,Willowdale West,43.782736,-79.442259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,4,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259,Fast Food Restaurant,Sushi Restaurant,Restaurant,Vietnamese Restaurant,Cuban Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant
29,4,M3J,North York,"Northwood Park, York University",43.767980,-79.487262,Caribbean Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant


In [47]:
# create updated map
map_clusters2 = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, neigh, cluster,one,two,three in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], 
                                                       toronto_merged['Neighborhood'], toronto_merged['Cluster Labels'],
                                                       toronto_merged['1st Most Common Restaurant'],toronto_merged['2nd Most Common Restaurant'],toronto_merged['3rd Most Common Restaurant']  
                                                      ):
    label = folium.Popup(' Cluster ' + str(cluster) + ' || '+' Borough - '+ str(poi) + ' || ' + ' Neighborhood(s) - ' + str(neigh) + ' || ' + 'Top 3 Cuisines: ' + 
                         str(one) + ', ' + str(two) + ', ' + str(three), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters2)
       
map_clusters2